In [1]:
import hashlib
import secrets
import os
import bcrypt


In [2]:
def sha256_hash(data: bytes) -> str:
    """Return hex SHA-256 digest of data bytes."""
    return hashlib.sha256(data).hexdigest()

def hash_without_salt(password: str) -> str:
    """Deterministic hash: same password -> same hash every time."""
    return sha256_hash(password.encode('utf-8'))

def hash_with_salt(password: str, salt: bytes = None) -> tuple:
    """
    Create a random salt if not provided, return (salt, hash_hex).
    We prepend the salt to the password before hashing (common pattern).
    """
    if salt is None:
        salt = secrets.token_bytes(16)  # 128-bit random salt
    combined = salt + password.encode('utf-8')
    return salt, sha256_hash(combined)

def hash_with_salt_and_pepper(password: str, salt: bytes = None, pepper: bytes = None) -> tuple:
    """
    Pepper should be a secret stored outside the DB. We combine:
    salt + password + pepper -> hash.
    Returns (salt, hash_hex). Caller must keep pepper secret.
    """
    if salt is None:
        salt = secrets.token_bytes(16)
    if pepper is None:
        raise ValueError("Pepper is required for this function.")
    combined = salt + password.encode('utf-8') + pepper
    return salt, sha256_hash(combined)


In [3]:
password = "user123password"

# 1) Without salt: run twice
hash1 = hash_without_salt(password)
hash2 = hash_without_salt(password)  # same password -> same hash

print("1) WITHOUT SALT (deterministic)")
print("Hash 1:", hash1)
print("Hash 2:", hash2)
print("Same?:", hash1 == hash2)
print("-" * 60)

# 2) WITH SALT: generate salt each time
salt_a, salted_hash_a = hash_with_salt(password)
salt_b, salted_hash_b = hash_with_salt(password)  # new random salt

print("2) WITH SALT (random salt each time)")
print("Salt A (hex):", salt_a.hex())
print("Hash A:", salted_hash_a)
print("Salt B (hex):", salt_b.hex())
print("Hash B:", salted_hash_b)
print("Same salts?:", salt_a == salt_b)
print("Same hashes?:", salted_hash_a == salted_hash_b)
print("-" * 60)

# 3) WITH SALT + PEPPER
# Simulate a secret pepper stored separately (attacker does NOT have this)
pepper = secrets.token_bytes(16)  # keep this secret outside DB
salt_c, hash_c = hash_with_salt_and_pepper(password, pepper=pepper)
salt_d, hash_d = hash_with_salt_and_pepper(password, pepper=pepper)  # different salt, same pepper

print("3) WITH SALT + PEPPER")
print("Pepper (hex) [SECRET; store outside DB]:", pepper.hex())
print("Salt C (hex):", salt_c.hex())
print("Hash C:", hash_c)
print("Salt D (hex):", salt_d.hex())
print("Hash D:", hash_d)
print("Same hashes?:", hash_c == hash_d)
print("-" * 60)

# 4) Simulate attacker who stole DB (hashes + salts) but DOES NOT have pepper
print("4) ATTACKER SCENARIO")
# Attacker tries to verify by hashing known password with the stored salt (but without pepper)
attacker_try = sha256_hash(salt_c + password.encode('utf-8'))  # attacker doesn't know pepper
print("Attacker computed (salt_c + password):", attacker_try)
print("Matches stored salted+pepper hash?:", attacker_try == hash_c)
print("-> Without the pepper the attacker cannot reproduce the salted+peppered hash.")
print("-" * 60)


1) WITHOUT SALT (deterministic)
Hash 1: 6384e02d9a47ac61c64950915c878ea54cc91a3d9c11d7497818017db8356d2a
Hash 2: 6384e02d9a47ac61c64950915c878ea54cc91a3d9c11d7497818017db8356d2a
Same?: True
------------------------------------------------------------
2) WITH SALT (random salt each time)
Salt A (hex): 13daebd3e530b34c25a4637b5e5eff06
Hash A: ab0cedc5ca9bf4f0bde06f5fb610d8247878e74723d449f22f001ebb3ad02a31
Salt B (hex): b6462d0461bd6463ea2d2a0eae652652
Hash B: 8491768fc3678311146eac7d826756b8c96eed6cb6be01f22972c73f7e026729
Same salts?: False
Same hashes?: False
------------------------------------------------------------
3) WITH SALT + PEPPER
Pepper (hex) [SECRET; store outside DB]: daa93b2c2fd9ce0db9149efe3424d09c
Salt C (hex): 9b80ee6968bfbb7517f7ee087d353b7a
Hash C: 38b51195e92a9973e6be6920f8c250e9ffc19277899f5564b9a7ace5dcb0abbc
Salt D (hex): 7e46c59c23414c123ce04bc5f7bf44d1
Hash D: cd7141bb5efe013e72606dc61d6b0c3de771753dec1981313659398b7a9d38d4
Same hashes?: False
----------------

In [4]:
# bcrypt automatically generates a salt and embeds it in the hash output
pw = password.encode('utf-8')
bcrypt_hash1 = bcrypt.hashpw(pw, bcrypt.gensalt())  # different salt each time
bcrypt_hash2 = bcrypt.hashpw(pw, bcrypt.gensalt())

print("BCRYPT DEMO")
print("bcrypt hash 1:", bcrypt_hash1.decode())
print("bcrypt hash 2:", bcrypt_hash2.decode())
print("Same?:", bcrypt_hash1 == bcrypt_hash2)
print()
# Verification using bcrypt.checkpw
print("bcrypt.checkpw with original password and hash1:", bcrypt.checkpw(pw, bcrypt_hash1))


BCRYPT DEMO
bcrypt hash 1: $2b$12$Af5./2Etl6g0Y9/bBeL/KOcxbXdjhBNfb6mZf9YzYYm1MIfGuuAvi
bcrypt hash 2: $2b$12$HiYGQq1SiUbdNLl6R.PfZu42mbOOQQS15Ronn/4gK1CX.w8bORDsa
Same?: False

bcrypt.checkpw with original password and hash1: True
